<a href="https://colab.research.google.com/github/simran487/ai-study-notes/blob/main/app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
!pip install streamlit pyngrok PyMuPDF transformers


In [87]:
%%writefile app.py
import streamlit as st
import fitz  # PyMuPDF
from transformers import pipeline
import torch

device = 0 if torch.cuda.is_available() else -1
summarizer = pipeline(
    "summarization",
    model="sshleifer/distilbart-cnn-12-6",  # lighter model for speed
    device=device
)

def extract_text_from_pdf(pdf_file):
    """Extract text from uploaded PDF"""
    text = ""
    pdf = fitz.open(stream=pdf_file.read(), filetype="pdf")
    for page in pdf:
        text += page.get_text()
    return text

st.title("📚 AI Study Notes Generator (Free)")

uploaded_pdf = st.file_uploader("Upload your PDF", type=["pdf"])

if uploaded_pdf:
    with st.spinner("Extracting text from PDF..."):
        text_data = extract_text_from_pdf(uploaded_pdf)

    if len(text_data.strip()) == 0:
        st.error("No readable text found in the PDF.")
    else:
        st.success("Text extracted successfully! Generating notes...")

        chunks = [text_data[i:i+1000] for i in range(0, len(text_data), 1000)]
        notes = []
        for chunk in chunks:
            summary = summarizer(chunk, max_length=150, min_length=50, do_sample=False)
            notes.append(summary[0]['summary_text'])

        st.subheader("📌 AI-Generated Study Notes")
        st.write("\n\n".join(notes))

        # Download button
        notes_text = "\n\n".join(notes)
        st.download_button("Download Notes", notes_text, file_name="study_notes.txt")


Overwriting app.py


In [89]:
from pyngrok import ngrok

# Add your ngrok token
!ngrok config add-authtoken 2vIpNZRfIPSOhR3dyxeZFic96c7_4sDeaEH5ZqHvtaQ6pczMc

# Kill any existing tunnels
ngrok.kill()

# Start Streamlit app in background
!streamlit run app.py &>/dev/null&

# Create public link
public_url = ngrok.connect(8501)
print("Click this link to open the app:", public_url)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Click this link to open the app: NgrokTunnel: "https://7c6a6318d301.ngrok-free.app" -> "http://localhost:8501"
